In [67]:
from transformers import (pipeline, set_seed,
    T5ForConditionalGeneration, T5Tokenizer
)
from utils import load_dataset_from_disk, get_reduced_dataset
from datasets import Dataset
import pandas as pd
import importlib
import torch
import os
import config
from config import DATA, MODEL, SEED, TRAINER_ARGUMENTS

In [53]:
def filter_duplicates(dataset):
    "Basic filtering function."
    dup_answers = set()
    dedup_records = []
    for question in dataset:
        if question["question"] not in dup_answers:
            dup_answers.add(question["question"])
            dedup_records.append(question)
    return dedup_records

In [54]:
USE_GPU = MODEL["use_gpu"]
DEVICE = "cuda:0" if USE_GPU and torch.cuda.is_available() else "cpu"
set_seed(SEED)

dataset = load_dataset_from_disk(
    DATA["dataset_path"],
    DATA["train_file_path"],
    DATA["test_file_path"],
    DATA["validation_file_path"],
)
dataset = get_reduced_dataset(dataset, 500)
dataset = dataset["validation"]
dataset = filter_duplicates(dataset)
dataset = Dataset.from_list(dataset)

In [55]:
dataset

Dataset({
    features: ['context', 'question', 'answer', 'input_text'],
    num_rows: 163
})

## **Pre-trained T5-small model**

In [73]:
tokenizer = T5Tokenizer.from_pretrained(MODEL["model_name"], legacy=False)
model = T5ForConditionalGeneration.from_pretrained(
    MODEL["model_name"], cache_dir=MODEL["model_cache_path"]
)
model.to(DEVICE)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer,
    max_length=MODEL["hyperparams"]["max_label_len"],
    # temperature=0.7,
    # num_beams=4,
    # top_p=0.95,
    repetition_penalty=1.15,
    device = DEVICE
)

In [74]:
gen_text_def = pipe(dataset["input_text"])

In [75]:
def_records = []
for idx, question in enumerate(dataset):
    print(f"{idx=}")
    print(f"{question['input_text']=}")
    print(f"{question['answer']=}")
    print(f"{gen_text_def[idx]['generated_text']=}\n")
    question["idx"] = idx
    question["gen_answer"] = gen_text_def[idx]['generated_text']
    def_records.append(question)

idx=0
question['input_text']='question: Which NFL team represented the AFC at Super Bowl 50? context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'
question['answer']='Denver Broncos'
gen_text_def[idx]['generated_text']='Denver Broncos'

idx=1
question['input_text']='

## **Fine-tuned T5-small model**

In [76]:
model = T5ForConditionalGeneration.from_pretrained(
    os.path.join(TRAINER_ARGUMENTS["output_dir"], "best"),
    cache_dir=MODEL["model_cache_path"]
)
model.to(DEVICE)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer,
    max_length=MODEL["hyperparams"]["max_label_len"],
    # temperature=0.7,
    # num_beams=4,
    # top_p=0.95,
    repetition_penalty=1.15,
    device = DEVICE
)

In [77]:
gen_text = pipe(dataset["input_text"])

In [78]:
ft_records = []
for idx, question in enumerate(dataset):
    print(f"{idx=}")
    print(f"{question['input_text']=}")
    print(f"{question['answer']=}")
    print(f"{gen_text[idx]['generated_text']=}\n")
    question["idx"] = idx
    question["gen_answer"] = gen_text[idx]['generated_text']
    ft_records.append(question)

idx=0
question['input_text']='question: Which NFL team represented the AFC at Super Bowl 50? context: Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'
question['answer']='Denver Broncos'
gen_text[idx]['generated_text']='Denver Broncos'

idx=1
question['input_text']='ques

## **Comparison**

In [86]:
df_def = pd.DataFrame.from_records(def_records)
df_def = df_def[[
    "idx",
    "question",
    "context",
    "answer",
    "gen_answer"
]]


df_ft = pd.DataFrame.from_records(ft_records)
df_ft = df_ft[[
    "idx",
    "question",
    "context",
    "answer",
    "gen_answer"
]]

In [98]:
df_compare = df_def.merge(df_ft, left_on='idx', right_on='idx',
          suffixes=('_def', '_ft'))
df_compare.drop(columns=[
    "question_ft", "context_ft", "answer_ft", "context_def"
], inplace=True)

In [100]:
pd.set_option('display.max_rows', len(dataset))
pd.set_option('display.max_colwidth', 0)
df_compare

,idx,question_def,answer_def,gen_answer_def,gen_answer_ft
0,0,Which NFL team represented the AFC at Super Bowl 50?,Denver Broncos,Denver Broncos,Denver Broncos
1,1,Which NFL team represented the NFC at Super Bowl 50?,Carolina Panthers,Carolina Panthers,Carolina Panthers
2,2,Where did Super Bowl 50 take place?,"Santa Clara, California",San Francisco Bay Area,at Levi's Stadium
3,3,Which NFL team won Super Bowl 50?,Denver Broncos,American Football Conference,Carolina Panthers
4,4,What color was used to emphasize the 50th anniversary of the Super Bowl?,gold,gold,gold
5,5,What was the theme of Super Bowl 50?,"""golden anniversary""",gold-themed initiatives,gold-themed initiatives
6,6,What day was the game played on?,"February 7, 2016","February 7, 2016","February 7, 2016"
7,7,What is the AFC short for?,American Football Conference,Super Bowl L,American Football Conference
8,8,What does AFC stand for?,American Football Conference,American Football Conference,American Football Conference
9,9,What day was the Super Bowl played on?,"February 7, 2016","February 7, 2016","February 7, 2016"
